In [ ]:
# ------------------------------------------------ #
#                  IMPORT FUNCTIONS                #
# ------------------------------------------------ #
import os
import sys
import datetime

# Get the project root directory (going up one level from Notebooks)
project_root = os.path.dirname(os.getcwd())
# Add the Scripts directory to Python path so we can import modules from individual scripts files
scripts_dir = os.path.join(project_root, 'Scripts')
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

# Add functions here from Scripts folder
from download_ao_indices import ReturnAOFetch
from enso_indices import ReturnENSOFetch
from breakup_data import CalculateBreakupData
from bounding_box import CalculateBoundingBox
from raster_temps import CalculateMeanTemperature
from raster_temps import CalculateTemperatureAnomaly
from correlation_raster import CompareIndexAndTemperature


In [ ]:
# ------------------------------------------------ #
#                CLASS DEFINITION                  #
# ------------------------------------------------ #

class StudySite:
    def __init__(self,name):
        self.name = name
        self.breakup_data = None # Dictionary of integer years (e.g. 2000) with following data subkeys: zscore_index, anomaly_days, breakup_date
        self.bounding_box = None # List of 4 lat,lon float64 coordinate pairs for top left, top right, bottom left, bottom right corners of bounding box
        self.point_temps = None
        self.raster_temps = None
        self.raster_anomalies = None
        self.enso_correlations = None
        self.ao_correlations = None

    def calculate_breakup_data(self, ice_data_filepath, years_of_data):
        # Load ice index data for this site from the icedata spreadsheet, filtering by years_of_data and the site name
        self.breakup_data = CalculateBreakupData(years_of_data, ice_data_filepath, self.name)

        # Access the data in main by using Site.breakup_data[2000][zscore_index]

    def calculate_bounding_box(self, ice_data_filepath):
        # Create bounding box for this site based on lat,lon for each site in the ice data CSV
        self.bounding_box = CalculateBoundingBox(ice_data_filepath, self.name)

    def calculate_mean_raster_temps(self, years_of_data):
        self.raster_temps = CalculateMeanTemperature(years_of_data, self.bounding_box)

    def calculate_anomaly_raster_temps(self, years_of_data):
        self.raster_anomalies = CalculateTemperatureAnomaly(years_of_data, self.bounding_box)

    def calculate_ao_correlation(self, ao_indices):
        self.ao_correlations = CompareIndexAndTemperature(ao_indices)
        
    def calculate_enso_correlation(self, enso_indices):
        self.enso_correlations = CompareIndexAndTemperature(enso_indices)

In [ ]:
# ------------------------------------------------ #
#                  PARAMETERS                      #
# ------------------------------------------------ #

# Add data here from Data folder
enso_data_filepath = os.path.join(project_root, "Data", "ENSO_index.rtf")
ao_data_filepath = os.path.join(project_root, "Data", "monthly.ao.index.txt")
ice_data_filepath = os.path.join(project_root, "Data", "ice_data.csv")

# Choose study sites here
study_site_names = ["Stebbins", "Hooper Bay"]

# Generate a list of years from 2000 to 2022 - select years
years_of_data = list(range(2000, 2023))

In [ ]:
# ------------------------------------------------ #
#                  MAIN CODE                      #
# ------------------------------------------------ #

# Pull in ENSO data as dictionary of Datetime:Values on a MONTHLY scale from all years of data
# Format of this dictionary should go from March of year-1 to March of year (e.g. year 2000 is dictionary M1999-M2000) for each year
enso_indices = ReturnENSOFetch(years_of_data, enso_data_filepath)

# Pull in AO data as dictionary of Datetime:Values on a MONTHLY scale from all years of data
# Format of this dictionary should go from March of year-1 to March of year (e.g. year 2000 is dictionary M1999-M2000) for each year
ao_indices = ReturnAOFetch(years_of_data, ao_data_filepath)s

# Create list of StudySite objects
sites = [StudySite(name) for name in study_site_names]

# Process data for each site
for site in sites:
    # Calculate basic site data
    site.calculate_breakup_data(ice_data_filepath, years_of_data)
    site.calculate_bounding_box(ice_data_filepath)
    site.calculate_mean_raster_temps()
    site.calculate_anomaly_raster_temps()

    # Calculate correlations
    site.calculate_ao_correlation(ao_indices)
    site.calculate_enso_correlation(enso_indices)
    
    

In [ ]:
# ------------------------------------------------ #
#       CORRELATION AND PLOTTING (Placeholders)    #
# ------------------------------------------------ #

# Placeholder for correlation calculations (actual functions need to be defined)
# ao_correlation_values = CalculateAOCorrelation(ao_indices, ice_indices)
# enso_correlation_values = CalculateENSOCorrelation(enso_indices, ice_indices)

# Placeholder for charting (actual functions need to be defined)
# ChartENSOValues(enso_correlation_values)
# ChartAOValues(ao_correlation_values)

# Generate visualization charts
# ChartENSOValues(save_path=os.path.join(project_root, "Figures", f"enso_correlation_{site.name}.png"))
# ChartAOValues(save_path=os.path.join(project_root, "Figures", f"ao_correlation_{site.name}.png"))